In [ ]:
import requests,io
from zipfile import ZipFile
r = requests.get('https://github.com/pulkitt15/imdb-dataset/blob/main/imdb.zip?raw=true')

with ZipFile(io.BytesIO(r.content), 'r') as zip_ref:
    zip_ref.extractall('/content/Imdb-dataset/train')

In [ ]:
r = requests.get('https://github.com/nvntcs18/imdb-dataset/blob/main/test.zip?raw=true')

with ZipFile(io.BytesIO(r.content), 'r') as zip_ref:
    zip_ref.extractall('/content/Imdb-dataset/test')

In [ ]:
#!/usr/bin/env python3
# -- coding: utf-8 --
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer
import re
import os

nltk.download('stopwords')
tokenizer=ToktokTokenizer()

def review_to_words(text):
    soup = BeautifulSoup(text,"html.parser")
    text=soup.get_text()
    text = re.sub('\[[^]]*\]', '', text)
    text = re.sub(r"[^a-zA-Z]"," ",text)
    tokens = tokenizer.tokenize(text.lower())
    tokens = [token.strip() for token in tokens]
    ps=PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in tokens]
    stop=set(stopwords.words('english'))
    filtered_tokens = [token for token in stemmed_tokens if token not in stop]
    return ' '.join(filtered_tokens)

def get_train_data():
    filenames = []
    for _,_,file in os.walk('/content/Imdb-dataset/train/imdb/pos'):
        filenames = file
    x_train=[]
    y_train=[]
    for filename in filenames:
         with open('/content/Imdb-dataset/train/imdb/pos/'+filename, 'r') as f:
             corpus = f.read()
             x_train.append(corpus)
             y_train.append(1)
                    
    for _,_,file in os.walk('/content/Imdb-dataset/train/imdb/neg'):
        filenames = file
    for filename in filenames:
         with open('/content/Imdb-dataset/train/imdb/neg/'+filename, 'r') as f:
             corpus = f.read()
             x_train.append(corpus)
             y_train.append(0)
    return x_train,y_train

def get_test_data():
    filenames = []
    for _,_,file in os.walk('/content/Imdb-dataset/test/test01/pos'):
        filenames = file
    x_train=[]
    y_train=[]
    for filename in filenames:
         with open('/content/Imdb-dataset/test/test01/pos/'+filename, 'r') as f:
             corpus = f.read()
             x_train.append(corpus)
             y_train.append(1)
                    
    for _,_,file in os.walk('/content/Imdb-dataset/test/test01/neg'):
        filenames = file
    for filename in filenames:
         with open('/content/Imdb-dataset/test/test01/neg/'+filename, 'r') as f:
             corpus = f.read()
             x_train.append(corpus)
             y_train.append(0)
    return x_train,y_train

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(review_to_words(get()[0]))

bromwel high cartoon comedi it ran time program school life teacher my year teach profess lead believ bromwel high satir much closer realiti teacher scrambl surviv financi insight student see right pathet teacher pomp petti whole situat remind school i knew student i saw episod student repeatedli tri burn school i immedi recal high a classic line inspector i sack one teacher student welcom bromwel high i expect mani adult age think bromwel high far fetch piti


In [ ]:
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#from data_loader import get_data,review_to_words

X_train,y_train = get_train_data()
X_test,y_test = get_test_data()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)


for i in range(len(X_train)):
    X_train[i] = review_to_words(X_train[i])

for i in range(len(X_test)):
    X_test[i] = review_to_words(X_test[i])
    
#X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

cv=CountVectorizer(binary=False,ngram_range=(1,3), max_features=50000)

cv_train_reviews=cv.fit_transform(X_train)

cv_test_reviews=cv.transform(X_test)

In [ ]:
linear = svm.SVC(kernel='linear')
linear_fit = linear.fit(cv_train_reviews, y_train)
print(linear)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
linear_predict = linear_fit.predict(cv_test_reviews)
print(linear_predict)

linear_score = accuracy_score(y_test,linear_predict)
print("linear_score :",linear_score)

linear_report=classification_report(list(le.inverse_transform(y_test)),list(le.inverse_transform(linear_predict)),target_names=['pos','neg'])
print(linear_report)

[1 1 1 ... 0 0 0]
linear_score : 0.86172
              precision    recall  f1-score   support

         pos       0.86      0.87      0.86     12500
         neg       0.87      0.86      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



In [ ]:
linear_train_predict = linear_fit.predict(cv_train_reviews)
print(linear_train_predict)

linear_train_score = accuracy_score(y_train,linear_train_predict)
print("linear_train_score :",linear_train_score)


In [ ]:
cm_bow=confusion_matrix(y_test,linear_predict,labels=[1,0])
print(cm_bow)

[[10704  1796]
 [ 1661 10839]]
